# Transformata Fouriera dla obrazów cyfrowych. Filtracja w dziedzinie częstotliwości.

## Cel ćwiczenia

- Zapoznanie z wykorzystaniem transformaty Fouriera w przetwarzaniu obrazów cyfrowych.
- Zapoznanie z pojęciem F-obrazu (amplitudy i fazy).
- Zapoznanie z własnościami transformaty Fouriera.
- Zapoznanie z filtracją w dziedzinie częstotliwości.

Na jednym z poprzednich ćwiczeń zetknęliśmy się z pojęciem konwolucji.
Przykładem może być filtracja dolno i górnoprzepustowa.
Operacja ta odpowiada mnożeniu w dziedzinie częstotliwości zgodnie z zależnością:

\begin{equation}
\mathcal{F}(g(x,y)*h(x,y)) = \mathcal{F}(g(x,y)) \cdot \mathcal{F}(h(x,y))
\end{equation}

gdzie: $\mathcal{F}$ oznacza transformatę Fouriera, a $*$ jest splotem.

Operacja filtracji w dziedzinie częstotliwości może okazać się bardziej efektywna, jeżeli operacje $fft$ i $ifft$ (odpowiednio szybka transformata Fouriera -- *fast Fourier transform* -- oraz odwrotna szybka transformata Fouriera -- *inverse fast Fourier transform*) zajmą mniej czasu niż klasyczna konwolucja (zazwyczaj ma to miejsce dla dużego obrazu, z dużą maską).

Sama filtracja w dziedzinie częstotliwości to mnożenie punktowe całego obrazu przez jedną maskę.

W przypadku filtracji w dziedzinie częstotliwości zakłada się, że obraz "zawija się" na brzegach, co może powodować pewne artefakty (zostanie to pokazane w trakcie ćwiczenia).

W dziedzinie częstotliwości "działają" tylko filtry liniowe.
Filtry medianowe, maksymalne, minimalne itp. nie mają swoich odpowiedników.

## Dwuwymiarowa transformata Fouriera

1. Wczytaj plik "dwieFale.bmp" w skali szarości.
Jest to obraz powstały na podstawie następującej zależności:

\begin{equation}
L(m, n) = 128 + 127 \cdot \cos(\frac{2\pi m}{32}+\frac{3\pi}{4}) \cdot \cos(\frac{2\pi n}{8}-\frac{\pi}{2})
\end{equation}<br>

gdzie: $m$ i $n$ są odpowiednio numerami wierszy i kolumn.

2. Do realizacji dwuwymiarowej transformaty Fouriera służy funkcja `cv2.dft`.
Ustaw flagę `flags=cv2.DFT_COMPLEX_OUTPUT`.
Wykonaj transformatę na wczytanym obrazie.
W ten sposób uzyskuje się tzw. F-obraz.

3. Najniższe częstotliwości znajdują się w lewym-górnym rogu obrazu.
Dla celów wizualizacji (ale też przetwarzania) często wykonuje się tzw. przesunięcie F-obrazu, które powoduje, że niskie częstotliwości przesuwane są do środka obrazu.
Wykorzystaj funkcję `np.fft.fftshift`.
Jako pierwszy argument podaj wynik transformaty Fouriera.
Jako drugi argument podaj numery osi, wzdłuż których należy wykonać operację.
Pierwsza oś odnosi się do wierszy obrazu.
Druga oś odnosi się do kolumn obrazu.
Trzecia oś to część rzeczywista (`[:, :, 0]`) lub urojona (`[:, :, 1]`).
W naszym przypadku argument powinien wyglądać tak `[0,1]`.

4. Wyświetl wynik transformaty.
Na wspólnym wykresie umieść obraz oryginalny, amplitudę i fazę F-obrazu.
Amplitudę i fazę wyznacz za pomocą funkcji `cv2.cartToPolar`.
Pierwszym argumentem funkcji jest część rzeczywista wyniku, a drugim urojona.
Uwaga. W razie wątpliwości proszę sprawdzić rozmiary rezultatu transformaty Fouriera oraz przesunięcia.

5. Dla wizualizacji oblicz logarytm dziesiętny amplitudy: `ALog = np.log10(A + 1)`.
Wyświetl go zamiast amplitudy na poprzednim wykresie.

6. Wczytaj obrazy *kolo.bmp*, *kwadrat.bmp*, *kwadrat45.bmp*, *trojkat.bmp*.
Czy analizując F-obraz można coś powiedzieć o kierunku krawędzi obiektów?

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np
import math

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/'

fileNames = ["dwieFale.bmp", "kolo.bmp", "kwadrat.bmp", "kwadrat45.bmp", "kwadratKL.bmp", "kwadratS.bmp", "kwadratT.bmp" ,"lena.bmp", "trojkat.bmp", "literki.bmp", "wzorA.bmp"]
for fileName in fileNames:
    filePath = 'img/' + fileName
    if not os.path.exists(filePath):
        r = requests.get(url + fileName, allow_redirects=True)
        open(filePath, 'wb').write(r.content)

In [ ]:
def show(imgs, titles=None):
    if len(imgs) == 1:
        plt.figure(figsize=(10, 10))
        plt.imshow(imgs[0], cmap='gray')
        if titles is not None:
            plt.title(titles[0])
        plt.axis('off')
        plt.show()
        return
        
    _, axis = plt.subplots(1, len(imgs), figsize=(5*len(imgs), 5))
    for i, ax in enumerate(axis):
        ax.imshow(imgs[i], cmap='gray')
        if titles is not None:
            ax.set_title(titles[i])
        ax.axis('off')
    
    plt.show()

In [ ]:
def analyze(img):
    f_img = cv2.dft(img.astype('float'), flags=cv2.DFT_COMPLEX_OUTPUT)
    f_img_shift = np.fft.fftshift(f_img, [0, 1])

    A, phase = cv2.cartToPolar(f_img[:, :,0], f_img[:,:,1])
    ALog = np.log10(A + 1)

    A_shift, phase_shift = cv2.cartToPolar(f_img_shift[:, :,0], f_img_shift[:,:,1])
    ALog_shift = np.log10(A_shift + 1)
    
    fig = plt.figure(layout='constrained', figsize=(10, 5))
    subfigs = fig.subfigures(1, 2)

    axsLeft = subfigs[0].subplots(1, 1)
    axsLeft.imshow(img, cmap='gray')
    axsLeft.set_title('Original image')
    axsLeft.axis('off')

    axsRight = subfigs[1].subplots(2, 2, sharex=True)
    axsRight[0, 0].imshow(ALog, cmap='gray')
    axsRight[0, 0].set_title('Fourier transform')
    axsRight[0, 0].axis('off')
    
    axsRight[1, 0].imshow(phase, cmap='gray')
    axsRight[1, 0].set_title('Phase')
    axsRight[1, 0].axis('off')
    
    axsRight[0, 1].imshow(ALog_shift, cmap='gray')
    axsRight[0, 1].set_title('Fourier transform (shifted)')
    axsRight[0, 1].axis('off')
    
    axsRight[1, 1].imshow(phase_shift, cmap='gray')
    axsRight[1, 1].set_title('Phase (shifted)')
    axsRight[1, 1].axis('off')
    
    plt.show()

In [ ]:
analyze(cv2.imread('img/dwieFale.bmp', cv2.IMREAD_GRAYSCALE))

7. Sprawdź (empirycznie) poprawność stwierdzenia:
>Dwuwymiarowa transformata Fouriera jest złożeniem dwóch transformat jednowymiarowych (wykonanych np. najpierw wierszowo, a później kolumnowo). Jednowymiarowa transformata realizowana jest za pomocą funkcji fft (z bibloteki Numpy).
>
Wykonaj najpierw transformatę po wierszach: `FRow = np.fft.fft(I, axis=0)`.
Następnie po kolumnach: `FCol = np.fft.fft(FRow, axis=1)`.
Numpy zwraca wynik jako tablicę liczb zespolonych.
Część rzeczywistą można otrzymać w następujący sposób: `FCol.real`, a urojoną: `FCol.imag`.
Porównaj tak uzyskany wynik z rezultatem działania funkcji `cv2.dft`.
Można to zrobić wizualnie lub z wykorzystaniem funkcji `cv2.absdiff`.

In [ ]:
def comapre_tronsform(img):
    # Perform 2D Fourier Transform using cv2.dft
    F_cv2 = cv2.dft(img.astype('float'), flags=cv2.DFT_COMPLEX_OUTPUT)

    # Perform 1D Fourier Transform along rows and columns using numpy
    FRow = np.fft.fft(img, axis=0)
    FCol = np.fft.fft(FRow, axis=1)

    # Compare the results using cv2.absdiff
    diff_real = cv2.absdiff(F_cv2[:, :, 0], FCol.real)
    diff_imag = cv2.absdiff(F_cv2[:, :, 1], FCol.imag)
    print(f'difference (real)\tMAX: {np.max(diff_real)}\t MEAN: {np.mean(diff_real)}')
    print(f'difference (imag)\tMAX: {np.max(diff_imag)}\t MEAN: {np.mean(diff_imag)}')

    # Visualize the results
    show([diff_real, diff_imag], ['Real difference', 'Imaginary difference'])

In [ ]:
comapre_tronsform(cv2.imread('img/dwieFale.bmp', cv2.IMREAD_GRAYSCALE))

## Własności dwuwymiarowej transformaty Fouriera

1. Zbadaj jak zmienia się F-obraz (amplituda i faza) podczas następujących operacji: translacja, rotacja, zmiana rozmiaru, kombinacja liniowa.
Wykorzystaj stworzony wcześniej kod.<br>
Uwaga. Należy użyć przygotowanych obrazów, a nie "generować" własne.
2. Do badania translacji wykorzystaj obrazy *kwadrat.bmp* i *kwadratT.bmp*.
3. Przy badaniu rotacji wykorzystaj obrazy *kwadrat.bmp* i *kwadrat45.bmp*.
4. Przy badaniu zmiany rozmiaru wykorzystaj obrazy *kwadrat.bmp* i *kwadratS.bmp*.
5. Przy badaniu kombinacji liniowej wykorzystaj obrazy *kwadrat.bmp*, *kwadrat45.bmp* i *kwadratKL.bmp*.

In [ ]:
img_filenames = ['kwadrat', 'kwadratS', 'kwadratT', 'kwadrat45', 'kwadratKL']

for img_filename in img_filenames:
    img = cv2.imread(f'img/{img_filename}.bmp', cv2.IMREAD_GRAYSCALE)
    analyze(img)

## Odwrotna dwuwymiarowa transformata Fouriera

1. Wykorzystaj stworzony wcześniej kod. Wybierz dowolny obraz np "kolo.bmp".
2. Przed realizacją odwrotnego przekszałcenia należy wykonać odwrotne przesunięcie.
Wykorzystaj funkcję `np.fft.ifftshift`.
Pierwszym argumentem jest wynik transformaty Fouriera.
Drugim argumentem są numery osi, wzdłuż których należy wykonać operację.
3. Wykonaj odwrotną transformatę Fouriera za pomocą funkcji `cv2.idft`.
Jako drugi argument przekaż następujące flagi: `flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT`.
Wynik może mieć małą część urojoną przez błędy numeryczne.
Aby pozbyć się tego efekty należy obliczyć amplitudę:
        imgIFFT = cv2.magnitude(ifft[:, :, 0], ifft[:, :, 1])
Następnie wynik należy zaokrąglić (`np.round`) i zrzutować do typu `uint8`.
4. Wyświetl wynik.
Sprawdź (wizualnie i poprzez odjęcie) czy obraz oryginalny i po przekształceniach są takie same.

In [ ]:
def reversed_dft(img):
    # Perform 2D Fourier Transform using cv2.dft
    f_img = cv2.dft(img.astype('float'), flags=cv2.DFT_COMPLEX_OUTPUT)
    f_img_shift = np.fft.fftshift(f_img, [0, 1])
    
    A_shift, phase_shift = cv2.cartToPolar(f_img_shift[:, :, 0], f_img_shift[:, :, 1])
    ALog_shift = np.log10(A_shift + 1)
    
    # Odwrotne przesunięcie
    f_img_unshifted = np.fft.ifftshift(f_img_shift, [0, 1])
    
    # Odwrotna transformata Fouriera
    img_reversed = cv2.idft(f_img_unshifted, flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
    img_reversed = cv2.magnitude(img_reversed[:, :, 0], img_reversed[:, :, 1])
    img_reversed = np.round(img_reversed).astype('uint8')
    
    show([img, ALog_shift, img_reversed], ['Original', 'Fourier transform', 'Inverse DFT'])

In [ ]:
reversed_dft(cv2.imread('img/kwadrat.bmp', cv2.IMREAD_GRAYSCALE))

## Filtracja obrazu w dziedzinie częstotliwości

1. Wczytaj obraz "lena.bmp" w skali szarości.
Wykonaj transformatę Fouriera.
Wykorzystaj stworzony poprzednio kod.
Wyświetl obraz oryginalny, amplitudę (w skali logarytmicznej) i fazę.

2. Przeprowadź filtrację dolnoprzepustową - usuń górne częstotliwości.
Dla F-obrazu po operacji przesunięcia (`fftshift`) niskie częstotliwości leżą w jego centrum.

3. Na początku stwórz filtr "kołowy", dolnoprzepustowy.
Należy wygenerować macierze opisujące przestrzeń w dziedzinie częstotliwości.
Ich rozmiar musi być taki sam jak rozmiar przetwarzanego obrazu.
```python
        lenaSize = I_Lena.shape
        FSpaceRows  = 2 * np.fft.fftshift(np.fft.fftfreq(lenaSize[0]))
        FSpaceRowsM = np.outer(FSpaceRows, np.ones([1, lenaSize[1]]))
        FSpaceCols  = 2 * np.fft.fftshift(np.fft.fftfreq(lenaSize[1]))
        FSpaceColsM = np.outer(np.ones([1, lenaSize[0]]), FSpaceCols)
```
Powyższy kod wygeneruje dwie znormalizowane macierze częstotliwości: *FSpaceRowsM* i *FSpaceColsM*.
Następnie należy wyznaczyć macierz zawierającą "odległość" od składowej stałej.
```python
        FreqR = np.sqrt(np.square(FSpaceRowsM) + np.square(FSpaceColsM))
```

Uwagi:
- funkcja `fftfreq` generuje wektor częstotliwości $[-0.5, 0.5]$ o określonym rozmiarze, przy czym układ wartości jest taki, że najpierw od 0 do 0.5, a potem od -0.5 do 0,
- operacja `fftshift` zmienia ten układ na $[-0.5, 0.5]$,
- mnożenie przez 2 ustala ostatecznie zakres na $[-1, 1]$,
- operacja `outer` to tzw. iloczyn zewnętrzy dwóch wektorów, w naszym przypadku powoduje, że wektor pionowy lub poziomy jest "powielany" odpowiednią liczbę razy.   
- sugeruje się, aby przyglądnąć się jak wygląda macierz `FreqR` - czy to w debugerze, czy poprzez wizualizację.

4. Teraz należy wybrać interesujący zakres.
Tu można zdefiniować typ filtru (dolno, górno, pasmowoprzepustowy).
```python
        FilterF = FreqR <= 0.1
```

Filtr należy zwizualizować:
```python
        figFilter = plt.figure()
        axsFilter = figFilter.add_subplot(projection='3d')
        axsFilter.plot_surface(FSpaceRowsM, FSpaceColsM, FilterF, rstride=1,
                               cstride=1, cmap=plt.get_cmap('gray'), linewidth=0)
        figFilter.show()
```

5. Wykonaj właściwą filtrację, czyli mnożenie F-obrazu przez filtr FilterF.
Trzeba pamiętać, że F-obraz ma 2 kanały (rzeczywisty i urojony).
By mnożenie było możliwe należy więc powielić filtr również na 2 kanały.
```python
        FilterF3 = np.repeat(FilterF[:, :, np.newaxis], 2, axis=2)
```

6. Wykonaj operację odwrotnego przesunięcia i odwrotnej transformaty.
Oblicz wartość bezwzględną wyniku.
Wykorzystaj funkcję `cv2.magnitude`.
Pierwszym argumentem jest część rzeczywista.
Drugim argumentem jest część urojona.
Wynik wyświetl.

7. Poeksperymentuj z rozmiarem filtru (promieniem).
Zaimplementuj filtr górnoprzepustowy (zmiana znaku przy warunku na odległość) oraz pasmowoprzepustowy (dwa warunki na promień połączone operatorem AND '&' ).
Wykonaj co najmniej trzy filtry i wyświetl wyniki.

8. W ten sposób zaimplementowana filtracja wprowadza pewne artefakty w postaci "pierścieni" wokół krawędzi.
Zapobiec temu zjawisku można poprzez odpowiednie "modelowanie" filtra.
W tym celu wykorzystać należy okna, np. Hamminga, Hanninga, Chebysheva (znane z przetwarzania sygnałów 1D).
Zagadnienie to jest tematem zadania domowego do tego ćwiczenia.

In [ ]:
def freq_filter(img, flag=None, threshold=0.5):
    f_img = cv2.dft(img.astype('float'), flags=cv2.DFT_COMPLEX_OUTPUT)
    f_img_s = np.fft.fftshift(f_img, [0, 1])
    A, phase = cv2.cartToPolar(f_img_s[:,:,0], f_img_s[:,:,1])
    ALog = np.log10(A+1)

    show([img, ALog, phase], ['Original', 'Fourier transform', 'Phase'])

    FSpaceRows  = 2 * np.fft.fftshift(np.fft.fftfreq(img.shape[0])) 
    FSpaceRowsM = np.outer(FSpaceRows, np.ones([1, img.shape[1]]))
    FSpaceCols  = 2 * np.fft.fftshift(np.fft.fftfreq(img.shape[1]))
    FSpaceColsM = np.outer(np.ones([1, img.shape[0]]), FSpaceCols)
    FreqR = np.sqrt(np.square(FSpaceRowsM) + np.square(FSpaceColsM))

    # Filter selection
    match flag:
        case 0 | 'dolnoprzepustowy':
            FilterF = FreqR <= threshold
        case 1 | 'górnoprzepustowy':
            FilterF = threshold < FreqR
        case 2 | 'pasmowoprzepustowy':
            low_threshold = threshold[0]
            high_threshold = threshold[1]
            FilterF = (low_threshold <= FreqR) & (FreqR <= high_threshold)
        case _:
            FilterF = FreqR

    FilterF3 = np.repeat(FilterF[:, :, np.newaxis], 2, axis=2)
    mul = f_img_s * FilterF3

    f_image_unshifted = np.fft.ifftshift(mul, [0, 1])
    reverse_image = cv2.idft(f_image_unshifted, flags=cv2.DFT_SCALE)
    reverse_image = cv2.magnitude(reverse_image[:, :, 0], reverse_image[:, :, 1])
    reverse_image = np.round(reverse_image).astype('uint8')
    
    fig = plt.figure(layout='constrained', figsize=(10, 5))
    figL, figR = fig.subfigures(1, 2)

    axsFilter = figL.add_subplot(projection='3d')
    axsFilter.plot_surface(FSpaceRowsM, FSpaceColsM, FilterF, rstride=1,
                           cstride=1, cmap=plt.get_cmap('gray'), linewidth=0)
    
    plots = figR.subplots(2, 1)
    plots[0].imshow(FreqR, cmap='gray')
    plots[0].axis('off')
    
    plots[1].imshow(FilterF, cmap='gray')
    plots[1].axis('off')
    
    fig.show()
    
    show([img, reverse_image, np.abs(img - reverse_image)])


In [ ]:
img = cv2.imread('img/lena.bmp', cv2.IMREAD_GRAYSCALE)
freq_filter(img, 'dolnoprzepustowy', 0.1)
freq_filter(img, 'górnoprzepustowy', 0.1)
freq_filter(img, 'pasmowoprzepustowy', (0.2, 0.4))
freq_filter(img)

## Implementacja wyszukiwania wzorca za pomocą FFT

1. Wczytaj w skali szarości i wyświetl obrazy *literki.bmp* i *wzorA.bmp*.

2. Wyznacz transformatę Fouriera obrazu *literki.bmp*.

3. Obróć drugi obraz o $180^\circ$.
Zastosuj funkcję `np.rot90`.
Pierwszym argumentem jest obracana macierz, a drugim liczba obrotów o $90^\circ$.

4. Należy wyznaczyć transformatę Fouriera obróconego obrazu w taki sposób, żeby miała ona taki sam rozmiar jak pierwszy obraz.
W tym celu należy zastosować *Zero Padding*.
Operacja ta polega na uzupełnieniu obrazu zerami do oczekiwanego rozmiaru.
Uzupełnij obraz zerami z **prawej** strony i z **dołu**.
W tym celu należy wykorzystać funkcję `cv2.copyMakeBorder`.
    - Pierwszym argumentem jest obraz wejściowy.
    - Drugim argumentem jest liczba wierszy u góry.
    - Trzecim argumentem jest liczba wierszy u dołu.
    - Czwartym argumentem jest liczba kolumn z lewej.
    - Piątym argumentem jest liczba kolumn z prawej.
    - Szóstym argumentem jest flaga typu wypełnienia.
    Dla stałej wartości podaj `cv2.BORDER_CONSTANT`.
    - Siódmym argementem jest wartość pikseli w ramce.
    Przekaż `value=0`.

5. Wyznacz transformatę Fouriera obrazu stworzonego w poprzednim punkcie.

6. Wyniki obu transformat należy przekonwertować do liczb zespolonych.
Obecnie jest to dwukanałowa macierz.
Pierwszy kanał odpowiada za część rzeczywistą.
Drugi kanał odpowiada za część urojoną.
Aby to osiągnąć wystarczy wykonać działanie:
        Complex = Real + Imag * 1j

7. Przemnóż ze sobą zespolone wyniki transformat.

8. Wynik należy powrotnie przekształcić do dwukanałowej macierzy.
Aby to zrobić wykonaj operację:
        CompMat = cv2.merge([np.real(Complex), np.imag(Complex)])

9. Wykonaj odwrotną transformatę Fouriera.
Dodaj flagę `flags=cv2.DFT_COMPLEX_INPUT`.

10. Oblicz wartość bezwzględną wyniku.

11. Wykonaj morfologiczną operację **Top-Hat**, by znaleźć maksima lokalne.
Operacja ta zostanie dokładnej wyjaśniona w jednym z kolejnych ćwiczeń.
W tym celu wykorzystaj operację:
        cv2.morphologyEx(correlation, cv2.MORPH_TOPHAT, np.ones((3, 3), np.uint8))

12. Wyświetl obok siebie obraz wejściowy i wynik wykonanych operacji.
Czy możesz wskazać położenie wzoru na podstawie drugiego obrazu?

In [ ]:
literki = cv2.imread('img/literki.bmp', cv2.IMREAD_GRAYSCALE)
wzor = cv2.imread('img/wzorA.bmp', cv2.IMREAD_GRAYSCALE)

f_literki = cv2.dft(literki.astype('float'), flags=cv2.DFT_COMPLEX_OUTPUT)
# fs_literki = np.fft.fftshift(f_literki, [0, 1])
wzor180 = np.rot90(wzor, 2)

down = literki.shape[0] - wzor180.shape[0]
right = literki.shape[1] - wzor180.shape[1]

wzor180_pad = cv2.copyMakeBorder(wzor180, 0, down, 0, right, cv2.BORDER_CONSTANT, value=0)
f_wzor = cv2.dft(wzor180_pad.astype('float'), flags=cv2.DFT_COMPLEX_OUTPUT)
# fs_wzor = np.fft.fftshift(f_literki, [0, 1])

literki_com = f_literki[:, :, 0] + f_literki[:, :, 1] * 1j
wzor_com = f_wzor[:, :, 0] + f_wzor[:, :, 1] * 1j
mul = literki_com * wzor_com
comp_mat = cv2.merge([np.real(mul), np.imag(mul)])
inverse = cv2.idft(comp_mat, flags=cv2.DFT_COMPLEX_INPUT)
inverse = cv2.magnitude(inverse[:, :, 0], inverse[:, :, 1])
abs_inverse = np.abs(inverse)
result = cv2.morphologyEx(abs_inverse, cv2.MORPH_TOPHAT, np.ones((3, 3), np.uint8))

show([literki, result])